In [ ]:
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install selenium

In [ ]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument("--headless")  # GUI nahi hai is liye headless mode chalayenge
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)
driver.get("https://www.gps-coordinates.net/satellite")

print("Website Open Ho Gayi!")  # Check ke liye
driver.quit()


In [ ]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class GoogleEarthScraper:
    def __init__(self):
        self.driver = None
        self.wait = None
        self.zoomed = False

    def initiate_browser(self):
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")  # Background mode
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        self.driver = webdriver.Chrome(options=options)
        self.wait = WebDriverWait(self.driver, 20)
        self.driver.get('https://www.gps-coordinates.net/satellite')
        self.driver.maximize_window()

        try:
            pop_up = self.driver.find_element(By.CSS_SELECTOR, '#CybotCookiebotDialogBodyButtonDecline')
            pop_up.click()
        except:
            pass

        time.sleep(2)

    def take_screenshot(self, img_path):
        time.sleep(2)  # Ensure page is fully rendered
        self.driver.save_screenshot(img_path)

    def adjust_zoom(self):
        try:
            zoom_in = self.driver.find_element(By.CSS_SELECTOR, "button[aria-label$='in']")

            # Increase zoom to capture a closer view
            for _ in range(5):  # Increased zoom level
                zoom_in.click()
                time.sleep(1)

        except Exception as e:
            print(f"Zoom error: {e}")

    def scrape(self, coordinates_list, output_folder):
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        for index, coordinates in enumerate(coordinates_list):
            lat, long = coordinates
            print(f"Processing {lat}, {long}")

            lat_field = self.wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#latitude")))
            lat_field.clear()
            lat_field.send_keys(lat)

            long_key = self.driver.find_element(By.CSS_SELECTOR, "#longitude")
            long_key.clear()
            long_key.send_keys(long)

            self.driver.find_element(By.CSS_SELECTOR, "button[onclick='codeLatLng(1)']").click()

            max_btn = self.wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-pressed='false']")))
            max_btn.click()

            self.adjust_zoom()

            time.sleep(5)
            img_path = f"{output_folder}/lahore_image_{index}.png"
            self.take_screenshot(img_path)
            max_btn.click()

            print(f"Saved: {img_path}")

    def quit_browser(self):
        self.driver.quit()

# Generate GPS coordinates for Lahore

def generate_lahore_coordinates():
    lat_start, lat_end = 31.3, 31.7  # Lahore Latitude Range
    lon_start, lon_end = 74.2, 74.6  # Lahore Longitude Range
    step = 0.05  # Step size for grid

    coordinates = []
    lat = lat_start
    while lat <= lat_end:
        lon = lon_start
        while lon <= lon_end:
            coordinates.append((str(round(lat, 4)), str(round(lon, 4))))
            lon += step
        lat += step
    return coordinates

# Example Usage:
scraper = GoogleEarthScraper()
scraper.initiate_browser()
lahore_coordinates = generate_lahore_coordinates()
scraper.scrape(lahore_coordinates, "lahore_images")
scraper.quit_browser()
